1. Log a new model on MLflow and connect to AzureML
2. Insert ???
3. Deploy logged model as stream endpoint
4. Get AzureML model info from model_id (get endpoint_name from model_id)
5. Call endpoint and make predictions given pd.DataFrame with cols: IMAGE_ID, BLOB_FILEPATH
6. Convert predictions to correct format (m_id, i_id, class_prob, predlabel) --> dic
7. Insert predictions into predictions table

In [13]:
import json
import sys
from tqdm.auto import trange, tqdm
import concurrent.futures
import yaml
import os
from json import JSONEncoder
import requests

import numpy as np
import pandas as pd
import cv2
import imageio

from azure.storage.blob import BlobServiceClient, ContainerClient, BlobClient
from azure.identity import DefaultAzureCredential
from azure.ai.ml import MLClient
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model as AzureMLModel,
    Environment,
    CodeConfiguration,
)
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core import Workspace, Model, Experiment, Run
from azureml.core.model import InferenceConfig

import tensorflow as tf
from tensorflow.keras.models import Model as KerasModel, model_from_json
from tensorflow.keras.optimizers import Adam
from keras.callbacks import Callback

import mlflow
import mlflow.keras

# from model_serving import prediction
# from model_serving import mlflow_log
# with open("./model_serving/config.yaml") as f:
#     config = yaml.load(f, Loader=yaml.FullLoader)

#### Log a new model on MLflow and connect to AzureML

In [ ]:
model = # load_model that was trained using sql_utils.get_train_df() ????
ml_client = MLClient.from_config(credential=DefaultAzureCredential())
run_name = 'basemodel' # change each time obv

mlflow_log.log_model(model, ml_client, run_name) 

#### Now insert ???

In [ ]:
models = pd.DataFrame(app_utils.get_models())
model_id = models.m_id.max() + 1
model_name = run_name
data = [{'model_name': model_name, 
         'm_id': model_id}]

#### Deploy logged model as stream endpoint

In [ ]:
# how to add a tag to model to associate with model_id

scoring_file = './model_serving/score.py' # update this
deploy.deploy(model_name, scoring_file)

# maybe
test_deployment.test_deployment(model_name)

#### Get AzureML model info from model_id (get endpoint_name from model_id)

In [8]:
ml_client = MLClient(subscription_id="156ffac2-0545-4d4e-aab3-f89b83635d04",
resource_group="defaultresourcegroup-wus2",
workspace_name="pivot", credential=DefaultAzureCredential())
ws = Workspace.from_config('../model_serving/config.json')

In [9]:
experiment_name = config['experiment_name']
run_name = 'basemodel'

experiment = Experiment(workspace=ws, name=experiment_name)
# run = Run(experiment=experiment, run_id=run_name)

# Get a list of all models in the experiment
# models = Model.list(workspace=ws, experiment_name=experiment_name)
runs = experiment.get_runs()
models = []
for run in runs:
    run_models = run.get_models()
    models.extend(run_models)

AttributeError: 'Run' object has no attribute 'get_models'

In [10]:
experiment

Name,Workspace,Report Page,Docs Page
adt-pivot,pivot,Link to Azure Machine Learning studio,Link to Documentation


In [11]:
# get_runs() returns all runs in reverse chronological order (first is most recent)
for i in experiment.get_runs():
    print(i)

Run(Experiment: adt-pivot,
Id: 3ce8d4e5-6343-4d44-be1e-e9c99b3df968,
Type: None,
Status: Completed)
Run(Experiment: adt-pivot,
Id: 0f5b3a2b-fde9-401b-bdaa-31ed72ecc95c,
Type: None,
Status: Completed)


In [16]:
runs = []
for i in experiment.get_runs():
    runs.append(i)

In [23]:
runs[0].get_metrics()

{}

In [ ]:
for run in ml_client.jobs.list():
    print("Name",run.display_name)
    print("Type",run.type)
    print("Compute",run.compute)
    print("Status",run.status)

In [ ]:
endpoint_name = 

In [ ]:
# write a function
def get_model_info(model_id):
    return endpoint_name #, artifacts maybe

#### Call endpoint and make predictions given pd.DataFrame with cols: IMAGE_ID, BLOB_FILEPATH

In [8]:
# Need to figure out how to get 'azureml-model-deployment' from endpoint_name 
# Used in get_predictions()

In [3]:
# sample input
df = pd.DataFrame({'IMAGE_ID': [1, 2, 3], 
                   'BLOB_FILEPATH': ['D20160524T225721_IFCB107/IFCB107D20160524T225721P00213.png', 
                                     'D20160524T225721_IFCB107/IFCB107D20160524T225721P00575.png', 
                                     'D20160524T225721_IFCB107/IFCB107D20160524T225721P00561.png']})

In [4]:
df

,IMAGE_ID,BLOB_FILEPATH
0,1,D20160524T225721_IFCB107/IFCB107D20160524T2257...
1,2,D20160524T225721_IFCB107/IFCB107D20160524T2257...
2,3,D20160524T225721_IFCB107/IFCB107D20160524T2257...


In [12]:
# endpoint_name = get_model_info(m_id)
scoring_uri = config['scoring_uri'].format(endpoint_name='basemodel-endpoint')
api_key = config['api_key']

In [22]:
# df['cloud_urls'] = df.BLOB_FILEPATH.apply(lambda x: config['cloud_url'].format(filepath=x))
# df

,IMAGE_ID,BLOB_FILEPATH,cloud_urls
0,1,D20160524T225721_IFCB107/IFCB107D20160524T2257...,https://ifcb.blob.core.windows.net/naames/NAAM...
1,2,D20160524T225721_IFCB107/IFCB107D20160524T2257...,https://ifcb.blob.core.windows.net/naames/NAAM...
2,3,D20160524T225721_IFCB107/IFCB107D20160524T2257...,https://ifcb.blob.core.windows.net/naames/NAAM...


In [30]:
preds = prediction.predict(df, scoring_uri, api_key)
preds

<Response [200]>
[[0.0031898769084364176, 2.0788617355327332e-11, 8.588602213421836e-05, 0.00017344176012557, 8.921755068058701e-08, 0.0032710987143218517, 8.093526048469357e-06, 0.9575332403182983, 0.03550129383802414, 0.00023710746609140188], [0.007550970651209354, 5.126950100020622e-07, 0.00035927206045016646, 0.002601270331069827, 2.5344577352370834e-06, 0.20422795414924622, 0.0010179296368733048, 0.7188515663146973, 0.06120515614748001, 0.004182853270322084], [0.00013524151290766895, 1.200628485520383e-11, 1.257793337572366e-05, 0.00022038634051568806, 2.6521347535890527e-05, 0.00010345028567826375, 1.8474192131634481e-07, 0.390915185213089, 0.00037548429099842906, 0.6082109212875366]]


,i_id,probs
0,1,"[0.0031898769084364176, 2.0788617355327332e-11..."
1,2,"[0.007550970651209354, 5.126950100020622e-07, ..."
2,3,"[0.00013524151290766895, 1.200628485520383e-11..."


#### Convert predictions to correct format (m_id, i_id, class_prob, predlabel) --> dic

In [ ]:
classes = ['Chloro',
          'Cilliate',
          'Crypto',
          'Diatom',
          'Dictyo',
          'Dinoflagellate',
          'Eugleno',
          'Other',
          'Prymnesio',
          'Unidentifiable']

preds['class_prob'] = preds.probs.apply(lambda x: x[pd.Series(x).idxmax()])
preds['predlabel'] = preds.probs.apply(lambda x: classes[pd.Series(x).idxmax()])
preds['m_id'] = [m_id] * len(preds)

In [43]:
preds = get_predictions(df, 1)

In [44]:
preds

[{'i_id': 1,
  'class_prob': 0.9575332403182983,
  'predlabel': 'Other',
  'm_id': 1},
 {'i_id': 2,
  'class_prob': 0.7188515663146973,
  'predlabel': 'Other',
  'm_id': 1},
 {'i_id': 3,
  'class_prob': 0.6082109212875366,
  'predlabel': 'Unidentifiable',
  'm_id': 1}]

#### Getting test data, getting feature vectors

In [72]:
test = sql_utils.run_sql_query(
"""
WITH test_images AS (
    SELECT DISTINCT I_Id
    FROM metrics
    WHERE m_id = 0
)
SELECT I.I_ID, I.filepath
FROM images AS I
INNER JOIN test_images AS TI ON TI.I_Id = I.I_ID;
"""
)

In [88]:
test

,I_ID,filepath,cloud_urls
0,399781,NAAMES_ml/D20160524T225721_IFCB107/IFCB107D201...,https://ifcb.blob.core.windows.net/naames/NAAM...
1,399802,NAAMES_ml/D20160524T225721_IFCB107/IFCB107D201...,https://ifcb.blob.core.windows.net/naames/NAAM...
2,399803,NAAMES_ml/D20160524T225721_IFCB107/IFCB107D201...,https://ifcb.blob.core.windows.net/naames/NAAM...
3,399808,NAAMES_ml/D20160524T225721_IFCB107/IFCB107D201...,https://ifcb.blob.core.windows.net/naames/NAAM...
4,399815,NAAMES_ml/D20160524T225721_IFCB107/IFCB107D201...,https://ifcb.blob.core.windows.net/naames/NAAM...
...,...,...,...
99995,1382617,NAAMES_ml/D20151105T224631_IFCB107/IFCB107D201...,https://ifcb.blob.core.windows.net/naames/NAAM...
99996,1382633,NAAMES_ml/D20151105T224631_IFCB107/IFCB107D201...,https://ifcb.blob.core.windows.net/naames/NAAM...
99997,1382642,NAAMES_ml/D20151105T224631_IFCB107/IFCB107D201...,https://ifcb.blob.core.windows.net/naames/NAAM...
99998,1382654,NAAMES_ml/D20151105T224631_IFCB107/IFCB107D201...,https://ifcb.blob.core.windows.net/naames/NAAM...


In [79]:
# Load the model architecture from JSON file
json_file_path = './ml-workflow/model_ckpt/model-cnn-v1-b3.json'
with open(json_file_path, 'r') as json_file:
    loaded_model_json = json_file.read()

loaded_model = model_from_json(loaded_model_json)

# Load the model weights from H5 file
h5_file_path = './ml-workflow/model_ckpt/model-cnn-v1-b3.h5'
loaded_model.load_weights(h5_file_path)

# Remove the last layer
loaded_model = KerasModel(inputs=loaded_model.input, outputs=loaded_model.layers[-2].output)

# Print the summary
loaded_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 128, 128, 1)]     0         
                                                                 
 conv2d_10 (Conv2D)          (None, 128, 128, 32)      320       
                                                                 
 activation_12 (Activation)  (None, 128, 128, 32)      0         
                                                                 
 batch_normalization_11 (Bat  (None, 128, 128, 32)     128       
 chNormalization)                                                
                                                                 
 conv2d_11 (Conv2D)          (None, 128, 128, 16)      4624      
                                                                 
 activation_13 (Activation)  (None, 128, 128, 16)      0         
                                                             

In [89]:
test['cloud_urls'] = test.filepath.apply(lambda x: "https://ifcb.blob.core.windows.net/naames/{filepath}".format(filepath=x))

cloud_urls = df.cloud_urls.values
data = []
for c_url in cloud_urls:
    data.append(preprocess_input(np.expand_dims(imageio.v2.imread(c_url), axis=-1)))

In [97]:
data[0].shape

(128, 128, 1)

In [98]:
loaded_model.predict(data[:3])

ValueError: in user code:

    File "/Users/aditis/opt/anaconda3/lib/python3.9/site-packages/keras/engine/training.py", line 2041, in predict_function  *
        return step_function(self, iterator)
    File "/Users/aditis/opt/anaconda3/lib/python3.9/site-packages/keras/engine/training.py", line 2027, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/aditis/opt/anaconda3/lib/python3.9/site-packages/keras/engine/training.py", line 2015, in run_step  **
        outputs = model.predict_step(data)
    File "/Users/aditis/opt/anaconda3/lib/python3.9/site-packages/keras/engine/training.py", line 1983, in predict_step
        return self(x, training=False)
    File "/Users/aditis/opt/anaconda3/lib/python3.9/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/Users/aditis/opt/anaconda3/lib/python3.9/site-packages/keras/engine/input_spec.py", line 216

In [62]:
import importlib
importlib.reload(sql_utils)

<module 'utils.sql_utils' from '/Users/aditis/Documents/DATA590/PIVOT/PIVOT/utils/sql_utils.py'>

In [61]:
from utils import sql_utils

In [ ]:
"https://ifcb.blob.core.windows.net/naames/{filepath}"

In [65]:
sql_utils.get_images_to_metrize(1, 1)

DatabaseError: (2812, b"Could not find stored procedure 'GENERATE_IMAGES_TO_METRIZE'.DB-Lib error message 20018, severity 16:\nGeneral SQL Server error: Check messages from the SQL Server\n")